# Part 1: Creating the Data Frame

In [2]:
#Import Necessary Libraries
import pandas as pd
import numpy as np
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [3]:
#Scrape table from webpage
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipedia = urlopen(url)
soup = BeautifulSoup(wikipedia, 'html.parser')

In [4]:
#Create Array to hold data and read table
torontotb = soup.find_all('table')

postalcodes = []
boroughs = []
neighborhoods = []


for table in torontotb:
    rows = table.find_all('tr')
    
    for row in rows:
        cells = row.find_all('td')
        if len(cells) == 3:
            postalcode = cells[0]
            postalcodes.append(postalcode.text.strip())
            
            borough = cells[1]
            boroughs.append(borough.text.strip())
            
            neighborhood = cells[2]
            neighborhoods.append(neighborhood.text.strip())



In [13]:
#Populate Pandas Dataframe
toronto_df = pd.DataFrame(postalcodes, columns = ['Postal Code'])
toronto_df['Boroughs'] = boroughs
toronto_df['Neighborhoods'] = neighborhoods
toronto_df.head()

,Postal Code,Boroughs,Neighborhoods
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [14]:
# Remove all rows with Not assigned boroughs
toronto_df = toronto_df[toronto_df['Boroughs'] != 'Not assigned'].reset_index(drop=True)
toronto_df.head()

,Postal Code,Boroughs,Neighborhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [15]:
toronto_df.shape

(103, 3)

# Part 2: Retrieving the Latitude and Longitude for each coordinate

In [16]:
geo = pd.read_csv('https://cocl.us/Geospatial_data')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
toronto_geo = pd.merge(toronto_df, geo, on = 'Postal Code', how = 'left')
toronto_geo.head(10)

,Postal Code,Boroughs,Neighborhoods,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [18]:
toronto_geo.shape

(103, 5)